#  Quick introduction to NLP with Python using spacy

## Main goal: Remove headers and footers from emails

### December 2019
### Author: GradientIA TMC: $I^2 Team$ (Innovation and inivestigation Team)
### Group: Spanish publications: All the code comments will be posted in Spanish


* Ejemplo de uso de NLP vía librerías standar de Spacy. El objetivo es la eliminación de encabezados y firmas de los correos.
* Se procesan ficheros en formato .txt
* En un directorio de entrada tendremos los textos a analizar (ficheros de texto que comienzan por "mt_") y se guardarán en el mismo directorio los ficheros de salida (mismo nombre, finalizando con "_clean").  

---

* Se usan las librerías Numpy y Spacy con su funcionalidad en español
* Se utilizan los modelos estadísticos pre-entrenados en español. En este caso, es_core_news_sm

In [ ]:
import numpy as np
import spacy
import es_core_news_sm

from spacy.lang.es import Spanish

def convertir (file, text):   
    # Conversión del corpus (texto) a Sentencias
    sentencias = text.strip().split('\n')
    
    # Se prepara el fichero de salida
    new_file = file.name[0:-4] + "_clean.txt"
    
    _genera_texto(sentencias, new_file)

# Importante el umbral. No dudes en "jugar" con él para ajustarlo a tus necesidades 
# El umbral define los falsos positivos. 
# Un umbral demasiado bajo no generará fichero de salida.
# Un umbral demasiado alto, no eliminará cabeceras y firmas.
def _genera_texto(sentencias, fic, threshold=0.99):
    # Carga de modelos estadísticos pre-entrenados  
    etiquetado_sm = spacy.load('es_core_news_sm')

    # Se iteran las sentencias y, si no se contempla como cabecera o pié, se escribe en el fichero de salida.  
    with open(fic, "w") as n_file:
        for sentencia in sentencias:
            if _probabilidad (sentencia, etiquetado_sm) < threshold:
                n_file.write(sentencia)

# Cálculo simplificado de la probabilidad de que sea cabecera o pié
# Nos basamos en los verbos de cada sentencia.
# Se puede ajustar en base a los parámetros que cada uno quiera
def _probabilidad(sentencia, etiquetado):   
    # Obtención del etiquetado (tagger)
    doc = etiquetado(sentencia)

    verb_count = np.sum([token.pos_ != "VERB" for token in doc])
    if (len(doc)>0):
        return (float(verb_count) / len(doc))
    else:
        return (0.0)

In [ ]:
# Se barre el directorio de entrada pasado por parámtero, en donde se procesarán todos los ficheros
# de texto que queremos tratar. En este caso, sólo tratamos aquellos que comienzan por "mt_"
# Se borran todos aquellos que ya hayan sido procesados (finalizan con "_clean.txt")
# Entrada: Directorio a recorrer
# Salida: Lista de ficheros a tratar
import os

def devolverArchivos(directorio):
    lista = []
    for file in os.listdir(directorio):
        
        if (file[0:3] == 'mt_'):
            if (file[-9:] == 'clean.txt'):
                print (file[-9:])
                file_aux = directorio+file
                print ('Borrando...', file_aux)
                os.remove(file_aux)
            else:
                lista.append(os.path.join(directorio, file))    
    return (lista)

In [ ]:
# Directorio objetivo: download_mails
directorio = './download_mails/'

lista = []
lista = devolverArchivos(directorio)

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import codecs

for fichero in lista:
    print ('Tratando...', fichero)
    fichero_tratado = codecs.open (fichero, mode = 'r', encoding = 'utf-8')
    text = fichero_tratado.read()
    convertir (fichero_tratado, text)

print ('Terminado')